## Sumarry

First part on mini-project about Segmenting and Clustering Neighborhoods in Toronto. 

This notebook scraps data from wikipedia and pre-processes it

In [1]:
import pandas as pd
import numpy as np
import geocoder
#import folium


#!conda install -c anaconda beautifulsoup4
#!conda install -c conda-forge geocoder
#!conda install -c conda-forge folium=0.5.0 --yes

In [2]:
from bs4 import BeautifulSoup
import requests

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

#My_table = soup.find('table',{'class':'wikitable sortable jquery-tablesorter'})
My_table = soup.find('tbody')

postCodeList = []
boroughList = []
neighbourhoodList = []

for row in My_table.find_all('tr'):
    try:
        rowValue = row.find_all('td')
        postCodeList.append(rowValue[0].text)
        boroughList.append(rowValue[1].text)
        neighbourhoodList.append((rowValue[2].text)[:-1])
    except IndexError:pass
    

#print(neighbourhoodList)

    
df = pd.DataFrame()
df['PostalCode'] = postCodeList
df['Borough'] = boroughList
df['Neighbourhood'] = neighbourhoodList

df = df[df.Borough != 'Not assigned']
df = df.groupby(('PostalCode','Borough')).agg([('Neighbourhood', ', '.join)]).reset_index()

for index, row in df.iterrows():
    neighbourhood = row['Neighbourhood']

    if neighbourhood.item() == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

df_scrapped = pd.DataFrame()
df_scrapped['PostalCode']= df.iloc[:,0].copy()
df_scrapped['Borough']= df.iloc[:,1].copy()
df_scrapped['Neighbourhood']= df.iloc[:,2].copy()

In [ ]:
df_scrapped.head(20)

In [ ]:
df_scrapped.shape

In [3]:
!wget -O postal_code.csv https://cocl.us/Geospatial_data
postal_df = pd.read_csv('postal_code.csv')
postal_df = postal_df.rename(columns={'Postal Code':'PostalCode'})

df_complete  = pd.merge(df_scrapped, postal_df, on='PostalCode')

df_complete.head()


--2019-06-03 15:51:33--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-03 15:51:34--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-03 15:51:35--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-03 

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [4]:
df_scrapped.shape

(103, 3)